In [14]:
%load_ext autoreload
%autoreload 2

from cnn import CNN
from dyn_cnn import DynCNN
from util import Util
from patch import Patch
from fixer import Fixer
from uglify import Uglify

import cPickle as pickle
import os
import mahotas as mh
import numpy as np
import time

# import mlproof as mlp


from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
DOJO_USER_DATA_FOLDER = '/home/d/Dropbox/DOJOSTUDY/'
DOJO_USERS_IDS = [11,13,18,37,40,43,5,50,53,62]
DOJO_USERS = []
for d in DOJO_USERS_IDS:
    DOJO_USERS.append('S'+str(d)+'tif')
    
def read_user(user, slice):
    img = mh.imread(DOJO_USER_DATA_FOLDER+user+os.sep+'z='+str(slice).zfill(8)+'.tif')
    img = Util.crop_by_bbox(img, dojo_bbox)
    return Util.frame_image(img)

def dojo_user_vi(user, slice, gt):
    img = read_user(user, slice)
    return Util.vi(img, gt)

def fix_dojo_slice(cnn, slice, input_image, input_prob, input_rhoana, input_gold,t=0.):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 sureness_threshold=t,
                                                 oversampling=False)

    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, best_sureness, vi_s, surenesses

def fix_dojo_slice_user_simulated(cnn, slice, input_image, input_prob, input_rhoana, input_gold, error_rate=0,t=0):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses, good_fixes, bad_fixes = Fixer.splits_user_simulated(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 error_rate=error_rate,
                                                 sureness_threshold=t,                                                 
                                                 oversampling=False)
    
    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, good_fixes, bad_fixes, merge_pairs, vi_s

def threed_vi(gt, seg):
    total_vi = 0
    slice_vi = []    
    for i in range(10):
        current_vi = Util.vi(gt[i].astype(np.int64), seg[i].astype(np.int64))
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi

def user_threed_vi(gt, user):
    total_vi = 0
    slice_vi = []
    for i in range(10):
        current_vi = dojo_user_vi(user, i, gt[i])
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi
    
def cnn_threed_vi(cnn, input_image, input_prob, input_rhoana, input_gold):
    total_vi = 0
    slice_vi = []
    slice_sureness = []
    for i in range(10):
        current_vi, sureness = fix_dojo_slice(cnn, i, input_image, input_prob, input_rhoana, input_gold)
        total_vi += current_vi
        slice_vi.append(current_vi)
        slice_sureness.append(sureness)
    total_vi /= 10
    return total_vi, slice_vi, slice_sureness
        

In [2]:
input_image, input_prob, input_gold, input_rhoana, dojo_bbox = Util.read_dojo_data() 

In [3]:
cnn = CNN('mine_merged_large_7', 'patches_7th', ['image', 'prob', 'merged_array', 'larger_border_overlap'])

Loading ['/home/d/dojo_xp/nets/mine_merged_large_7/configuration.txt']
Loading /home/d/dojo_xp/nets/mine_merged_large_7/network_262.p


In [4]:
with open('/home/d/dojo_xp/data/merges2.p', 'rb') as f:
    merge_errors = pickle.load(f)

In [6]:
from uitools import UITools

In [7]:
#
# APPLY MERGE ERROR CORRECTION TO RHOANA
#
rhoana_after_merge_correction = np.array(input_rhoana)
# correct merge_errors if they are rated <.5
for me in merge_errors:
    pred = me[2]
    if pred < .5:
        z = me[0]
        label = me[1]
        border = me[3][0][1]
        a,b,c,d,e,f,g,h,i = UITools.get_merge_error_image(input_image[z], rhoana_after_merge_correction[z], label, border)        
        rhoana_after_merge_correction[z] = f
        print 'corrected', z

corrected 0
corrected 1
corrected 2
corrected 4
corrected 7
corrected 8
corrected 9


In [8]:
#
# APPLY MERGE ERROR CORRECTION TO RHOANA RANDOMLY
#
import random
rhoana_after_merge_correction_random = np.array(input_rhoana)
# correct merge_errors if they are rated <.5
for me in merge_errors:
    pred = me[2]
    if random.random() < .5:
        z = me[0]
        label = me[1]
        border = me[3][0][1]
        a,b,c,d,e,f,g,h,i = UITools.get_merge_error_image(input_image[z], rhoana_after_merge_correction_random[z], label, border)        
        rhoana_after_merge_correction_random[z] = f
        print 'corrected', z

corrected 8


In [9]:
# fix one slice a time with sureness threshold .95
vol_vi_s = []
for z in range(10):
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, input_image[z], input_prob[z], rhoana_after_merge_correction[z], input_gold[z], 
                                                 sureness_threshold=.95,
                                                 verbose=False)
    vol_vi_s.append(vi_s[-1])
    

In [10]:
np.median(vol_vi_s)

0.53607027266523577

In [11]:
vol_vi_s

[0.37714099477120566,
 0.4119345987453906,
 0.41743583312508914,
 0.4085899806629625,
 0.4501137754484592,
 0.6821418080964907,
 0.6761792610958874,
 0.6474979613212639,
 0.6873224969701655,
 0.6220267698820123]

In [15]:
# simulate user with error rate 0 with limit of N corrections
bigM, out_vol, global_volume = Fixer.splits_global(cnn, input_image, input_prob, rhoana_after_merge_correction, input_gold)

476 generated in 7.98501205444 seconds.
Grouped into 119 patches in 0.00241112709045 seconds.
456 generated in 9.01170992851 seconds.
Grouped into 114 patches in 0.00247406959534 seconds.
468 generated in 9.67066311836 seconds.
Grouped into 116 patches in 0.00281810760498 seconds.
484 generated in 9.60613203049 seconds.
Grouped into 121 patches in 0.00263690948486 seconds.
520 generated in 10.1628270149 seconds.
Grouped into 130 patches in 0.00283885002136 seconds.
488 generated in 9.88358092308 seconds.
Grouped into 122 patches in 0.00248599052429 seconds.
480 generated in 9.54860901833 seconds.
Grouped into 118 patches in 0.0018470287323 seconds.
476 generated in 9.00306010246 seconds.
Grouped into 117 patches in 0.00239992141724 seconds.
488 generated in 9.48145890236 seconds.
Grouped into 122 patches in 0.002366065979 seconds.
548 generated in 10.3332960606 seconds.
Grouped into 137 patches in 0.00253081321716 seconds.


In [18]:
# with 5 secs per correction
user_simulated_vi = threed_vi(input_gold, out_vol)
np.median(user_simulated_vi[1])

0.40237352272697935

In [19]:
user_simulated_vi

(0.4334672910251106,
 [0.3638747604452375,
  0.39741416580385813,
  0.3740647191814084,
  0.39954443665184947,
  0.40520260880210923,
  0.3903271023568733,
  0.5163180036974166,
  0.5326613477925966,
  0.5208193249023552,
  0.43444644061740156])

In [29]:
# random recommendations
bigM, out_vol, global_volume = Fixer.splits_global(cnn, input_image, input_prob, rhoana_after_merge_correction_random, input_gold, randomize=True)

464 generated in 8.26559686661 seconds.
Grouped into 116 patches in 0.00186085700989 seconds.
448 generated in 9.32174682617 seconds.
Grouped into 112 patches in 0.00222396850586 seconds.
456 generated in 9.37109494209 seconds.
Grouped into 113 patches in 0.00216293334961 seconds.
484 generated in 9.3007068634 seconds.
Grouped into 121 patches in 0.00239300727844 seconds.
508 generated in 10.3682589531 seconds.
Grouped into 127 patches in 0.00270414352417 seconds.
488 generated in 9.61003708839 seconds.
Grouped into 122 patches in 0.0024528503418 seconds.
480 generated in 9.77884984016 seconds.
Grouped into 118 patches in 0.00230503082275 seconds.
460 generated in 9.01172709465 seconds.
Grouped into 113 patches in 0.00247287750244 seconds.
488 generated in 9.67659783363 seconds.
Grouped into 122 patches in 0.0023250579834 seconds.
540 generated in 10.37352705 seconds.
Grouped into 135 patches in 0.00284790992737 seconds.


In [30]:
random_vi = threed_vi(input_gold, out_vol)
np.median(random_vi[1]) 

0.47222117404401454

In [31]:
random_vi

(0.4944449604498559,
 [0.43316548990430626,
  0.41386189487693503,
  0.40007574892515674,
  0.40067752909270915,
  0.4865130129098647,
  0.45792933517816437,
  0.5732451590105869,
  0.59937227290765,
  0.5868511567812682,
  0.5927580049119179])